In [1]:
from langchain.llms import HuggingFaceHub
from langchain.prompts import PromptTemplate

prompt = PromptTemplate.from_template("[INST]What is meaning of {word}[/INST]")

llm = HuggingFaceHub(
    repo_id="mistralai/Mistral-7B-Instruct-v0.3",
    model_kwargs={
        "max_new_tokens":250,
    }
)

llm.client.api_url = 'https://api-inference.huggingface.co/models/mistralai/Mistral-7B-Instruct-v0.3'

chain = prompt | llm

chain.invoke(
    {
        "word": "potato"
    }
)

/Users/samsepi0l/Documents/FullStack-GPT/env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


'A potato is a starchy, tuberous crop from the Solanum tuberosum species. It is native to the Andean region of South America and is a major food source worldwide. Potatoes are usually grown as an underground, bulbous structure (the tuber) and are a popular ingredient in many dishes due to their versatility. They can be boiled, baked, fried, or mashed and are often used in soups, stews, and salads. Potatoes are rich in carbohydrates, fiber, and vitamin C.'